### Load up selenium and the Chrome webdriver

In [6]:
pip install selenium

  Using cached selenium-4.27.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.27.1-py3-none-any.whl (9.7 MB)
Using cached trio-0.27.0-py3-none-any.whl (481 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting unins

In [80]:
import selenium
from selenium import webdriver
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [79]:
driver = webdriver.Chrome()

In [81]:
driver.get('https://generalssb-prod.ec.njit.edu/BannerExtensibility/customPage/page/stuRegCrseSched')

### In the left navigation, find the 16th entry ("CS") and click on it

#### This will cause the main content area to load all of the CS courses

In [85]:
driver.implicitly_wait(5)
left_link = driver.find_element(By.ID, 'pbid-subjListTableSubjectLink-18')
#18 is DS

In [86]:
cs_link = left_link.find_element(by=By.TAG_NAME, value='a')
cs_link.click()

In [87]:
my_span = driver.find_element(by=By.ID, value='pbid-courseListSectionDetailSections-0')

In [88]:
print(my_span)

<selenium.webdriver.remote.webelement.WebElement (session="d1b70027cc704dcdbb9238f53173a240", element="f.C8C481E7984116C668938FD3A1296B21.d.0265C92DCBF4D59705C7101FD14C7AFA.e.45")>


In [89]:
course_headers = my_span.find_elements(by=By.TAG_NAME, value="H4")

In [90]:
course_tables = my_span.find_elements(by=By.TAG_NAME, value="table")

In [91]:
print("I have {} headers and {} tables".format(len(course_headers), len(course_tables)))

I have 18 headers and 18 tables


In [92]:
def create_column_headings(table_headers):
    loc_columns = ['Course_name']
    for i in range(len(table_headers)):
        loc_columns.append(table_headers[i].text)
    return loc_columns

In [93]:
def create_course_listings(c_headers, c_tables):
    c_list = []
    for i in range(len(c_headers)):
        course_name = c_headers[i].text
#       collect the rows for each section of the course
        t_rows = c_tables[i].find_elements(by=By.CLASS_NAME, value='success')
#       for each row, get all of the cells
        for k in range(len(t_rows)):
            course_row = []
            course_row.append(course_name)
            t_cells = t_rows[k].find_elements(by=By.TAG_NAME, value='td')
            for l in range(len(t_cells)):
                course_row.append(t_cells[l].text)
            c_list.append(course_row)
    return c_list


In [94]:
# create column headings by using the 'th' cells of the first table
t_headers = course_tables[0].find_elements(by=By.TAG_NAME, value="th")
columns = create_column_headings(t_headers)

In [95]:
# use the list of course_headers and the list of course_tables to build
#   individual rows - one for each course/section
course_list = create_course_listings(course_headers, course_tables)
df = pd.DataFrame(course_list, columns=columns)
driver.quit()

In [96]:
df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,DS 340 - FUND. AND PRIN. OF DS,004,16164,MW,10:00 AM - 11:20 AM,CKB 120,Open,38,11,"Bhavsar, Jaini",Face-to-Face,3,Book,
1,DS 488 - INDEPENDENT STUDY IN DS,531,16403,,TBA,,Open,19,0,"Bader, David",Face-to-Face,3,Book,See department for permission to enroll.
2,DS 488 - INDEPENDENT STUDY IN DS,610,15926,,TBA,,Open,19,0,"Dasgupta, Aritra",Face-to-Face,3,Book,See department for permission to enroll.
3,DS 636 - DATA ANALYTICS WITH R PROGRAM,002,12240,M,1:00 PM - 3:50 PM,KUPF 210,Open,43,7,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
4,DS 636 - DATA ANALYTICS WITH R PROGRAM,004,12241,T,2:30 PM - 5:20 PM,TIER LECT 2,Open,43,15,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,DS 792B - PRE-DOCTORAL RESEARCH,606,12378,,TBA,,Open,19,0,"Wu, Chase",Face-to-Face,3,Book,See department for permission to enroll.
127,DS 792B - PRE-DOCTORAL RESEARCH,610,12379,,TBA,,Open,19,0,"Dasgupta, Aritra",Face-to-Face,3,Book,See department for permission to enroll.
128,DS 792B - PRE-DOCTORAL RESEARCH,616,12380,,TBA,,Open,19,0,"Phan, Hai",Face-to-Face,3,Book,See department for permission to enroll.
129,DS 792B - PRE-DOCTORAL RESEARCH,624,12381,,TBA,,Open,19,0,"Wang, Guiling",Face-to-Face,3,Book,See department for permission to enroll.


In [ ]:
df.describe()

In [ ]:
df['Max'] = df['Max'].astype(int)
df['Now'] = df['Now'].astype(int)

In [ ]:
df['Open'] = df['Max'] - df['Now']

In [ ]:
### potential rows to delete
        # skip Independent Study, Master's Project, Master's Thesis 
#        if course_name.startswith("CS 488") or course_name.startswith("CS 700B") \
#        or course_name.startswith("CS 701B") or course_name.startswith("CS 725") \
#        or course_name.startswith("CS 792") or course_name.startswith("CS 726") \
#        or course_name.startswith("CS 790A"):

In [ ]:
df = df[~df['Course_name'].str.contains('CS 488')]
df = df[~df['Course_name'].str.contains('CS 700B')]
df = df[~df['Course_name'].str.contains('CS 701B')]
df = df[~df['Course_name'].str.contains('CS 725')]
df = df[~df['Course_name'].str.contains('CS 792')]
df = df[~df['Course_name'].str.contains('CS 790A')]
df = df[~df['Course_name'].str.contains('CS 726')]

df

In [ ]:
df[['Instructor', 'Max', 'Now']][200:250]

In [ ]:
df[['Instructor','Open']].groupby(['Instructor']).sum().tail(50)


In [ ]:
df['Course_name'].unique()

In [126]:
df2 = df[['Instructor', 'Max', 'Now']].copy()

df2['Max'] = pd.to_numeric(df2['Max'], errors='coerce')
df2['Now'] = pd.to_numeric(df2['Now'], errors='coerce')

df2.loc[:, 'Seats Taken'] = df2['Max'] - df2['Now']

In [127]:
df2

,Instructor,Max,Now,Seats Taken
0,"Bhavsar, Jaini",38,11,27
1,"Bader, David",19,0,19
2,"Dasgupta, Aritra",19,0,19
3,"Bhavsar, Jaini",43,7,36
4,"Bhavsar, Jaini",43,15,28
...,...,...,...,...
126,"Wu, Chase",19,0,19
127,"Dasgupta, Aritra",19,0,19
128,"Phan, Hai",19,0,19
129,"Wang, Guiling",19,0,19
